# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
from silvhua import *

from openai import OpenAI
import os
from time import time

from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory.chat_message_histories import DynamoDBChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage

from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor

from langchain.agents import Tool


In [3]:

conversation_dict = dict()
system_message_dict = dict()
reply_dict = dict()
contacts = load_json('contacts.json', '../src/app/private')


# Iteration 1

In [4]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

## Examples

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 1
InboundMessage = "Can we book for Tuesday 9am?"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 2


> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'.", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

## 1.1

In [5]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage.
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 1.1
InboundMessage = "Can we book for Tuesday 9am?"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `foo-0` with `How long is the intro session`


foo{
  "response": "[ALERT HUMAN]",
  "alert_human": true
}

  {"response": "[ALERT HUMAN]", "alert_human": true}

> Finished chain.
Response time: 3.3201441764831543 seconds
Reply from `chat_with_chatbot`: {
  "response": "[ALERT HUMAN]",
  "alert_human": true
}

  {"response": "[ALERT HUMAN]", "alert_human": true}


In [6]:
InboundMessage = "I have cancer. Will you be able treat me?"
reply_dict[conversation_id] = dict()
question_id += 1

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 2


> Entering new AgentExecutor chain...

{
  "response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.",
  "alert_human": true
}

> Finished chain.
Response time: 2.0846633911132812 seconds
Reply from `chat_with_chatbot`: 
{
  "response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.",
  "alert_human": true
}


In [8]:
InboundMessage = "Can you help me rehabilitate from a herniated disc?"
question_id += 1

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `foo-0` with `Are you currently on a health and fitness journey or looking to start one?`


foo
    {
        "response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.",
        "alert_human": false
    }

    {
        "response": "I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.",
        "alert_human": false
    }

> Finished chain.
Response time: 4.526203632354736 seconds
Reply from `chat_with_chatbot`: 
    {
        "re

In [9]:
reply_dict[conversation_id][question_id]

{'input': 'Can you help me rehabilitate from a herniated disc?',
 'chat_history': [HumanMessage(content='How long is the intro session'),
  AIMessage(content='hey silvia this is jayson testing for brian')],
 'output': '\n    {\n        "response": "I\'m Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle.",\n        "alert_human": false\n    }\n\n    {\n        "response": "I\'m Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.",\n        "alert_human": false\n    }',
 'intermediate_steps': [(AgentActionMessageLog(

In [10]:
InboundMessage = "What is the cost of the program?"
question_id += 1

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `foo-0` with `How much does the program cost?`


foo{
  "response": "Ah yes. I say im willing to work within your  budget! Why dont we make sure I am the best fit first? Fair enough? Although my program is affordable I assure you it will definitely be the best investment you make in yourself!",
  "alert_human": false
}

    {
        "response": "Ah yes. I say im willing to work within your  budget! Why dont we make sure I am the best fit first? Fair enough? Although my program is affordable I assure you it will definitely be the best investment you make in yourself!",
        "alert_human": false
    }

> Finished chain.
Response time: 4.23983359336853 seconds
Reply from `chat_with_chatbot`: {
  "response": "Ah yes. I say im willing to work within your  budget! Why dont we make sure I am the best fit first? Fair enough? Although my program is affordable I assure you it will definitely be the best investment you 

In [11]:
InboundMessage = "Can you give me an Olympic lifting program to improve my technique?"
question_id += 1

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')

Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `foo-0` with `Can you give me an Olympic lifting program to improve my technique?`


foo{
  "response": "[ALERT HUMAN]",
  "alert_human": true
}

> Finished chain.
Response time: 2.3843140602111816 seconds
Reply from `chat_with_chatbot`: {
  "response": "[ALERT HUMAN]",
  "alert_human": true
}


## `parse_json_string`

In [14]:
import json

def parse_json_string(json_string):
    return json.loads(json_string)

reply_dict = parse_json_string(reply_text)
reply_dict

{'response': '[ALERT HUMAN]', 'alert_human': True}

In [16]:
reply_dict['alert_human']

True

# Iteration 2

In [18]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage.
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def fake_func(inp: str) -> str:
    return "foo"

tools = [
    Tool(
        name=f"foo-{i}",
        func=fake_func,
        description=f"a silly function that you can use to get more information about the number {i}",
    )
    for i in range(2)
]

conversation_id = 2
InboundMessage = "I have a herniated disc. Can you still coach me?"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `foo-0` with `Are you an AI or a human?`


foo{
  "response": "Are you an AI or a human?",
  "alert_human": true
}

    {
        "response": "Are you an AI or a human?",
        "alert_human": true
    }

> Finished chain.
Response time: 2.6719188690185547 seconds
Reply from `chat_with_chatbot`: {
  "response": "Are you an AI or a human?",
  "alert_human": true
}

    {
        "response": "Are you an AI or a human?",
        "alert_human": true
    }


# Iteration 3 update the function tools

In [19]:
def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage.
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ]
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def alert_human(str) -> str:
    return "[ALERT HUMAN]"

tools = [
    Tool(
        name=f"alert_human_function",
        func=alert_human,
        description=f"A function to use when you do not know how to respond to the InboundMessage because it cannot be answered from your instructions or documentation.",
    )
]

conversation_id = 3
InboundMessage = "I have a herniated disc. Can you still coach me?"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


Chat history length: 2


> Entering new AgentExecutor chain...

Invoking: `alert_human_function` with `Can I still coach someone with a herniated disc?`


[ALERT HUMAN]
{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}

{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}

> Finished chain.
Response time: 4.430384635925293 seconds
Reply from `chat_with_chatbot`: 
{"res

In [24]:
print(type(reply_text))
print(reply_text)
reply_text

<class 'str'>

{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}

{"response":"I'm Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}


'\n{"response":"I\'m Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}\n\n{"response":"I\'m Amanda - creator of the Strong and Sassy programme. I am a degree qualified nutritionist and coach with over 10 years experience. I educate women about the importance of health and wellness, so they can live a sustainable, healthy and balanced lifestyle. I help women become super confident, strong and happy within themselves.","alert_human":true}'

In [22]:
parse_json_string(reply_text)

JSONDecodeError: Extra data: line 4 column 1 (char 368)

# Iteration 4 Make sure JSON format is adhered to

In [31]:
# from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Chatbot_Response(BaseModel):
    response: str = Field(description="The response to the InboundMessage.")
    alert_human: bool = Field(description="Whether or not to alert a human to review the response.")

def create_system_message(
        business_name, prompts_filepath='../prompts',
        examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
        ):
    
    instructions = load_txt(f'{business_name}.md', prompts_filepath)
    examples = load_txt(f'{business_name}.txt', examples_filepath)
    document = load_txt(f'{business_name}_doc.md', doc_filepath)

    system_message = f"""# Stage 1

{instructions}

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage.
- "alert_human" (True or False): Whether or not to alert a human to review the response.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Make sure that if the question cannot be answered through the documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to ensure your response is concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def create_chatbot(contactId, system_message, tools, model="gpt-3.5-turbo-1106", verbose=True):

    llm = ChatOpenAI(
        temperature = 0,
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key'],
        model=model, 
        model_kwargs={"response_format": {"type": "json_object"}} # https://platform.openai.com/docs/guides/text-generation/json-mode  # https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.openai.ChatOpenAI.html?highlight=chatopenai#
        )
    message_history = DynamoDBChatMessageHistory(
        table_name="SessionTable", session_id=contactId,
        key={
            "SessionId": contactId,
            "type": 'ChatHistory',
            }
        )
    system_message = SystemMessage(
        content=(system_message),
        input_variables=['InboundMessage']
    )
    parser = SimpleJsonOutputParser(pydantic_object=Chatbot_Response)

    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[
            MessagesPlaceholder(variable_name='chat_history')
            ],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    agent_executor = AgentExecutor(
        agent=agent, tools=tools, 
        verbose=verbose, return_intermediate_steps=True
        )
    agent_info = {
        'agent': agent,
        'agent_executor': agent_executor,
        'chat_history': message_history.messages
    }
    return agent_info

def chat_with_chatbot(user_input, agent_info):
    start_time = time()
    print(f'Chat history length: {len(agent_info["chat_history"])}')
    chat_history = agent_info['chat_history']
    result = agent_info['agent_executor']({
        "input": user_input,
        "chat_history": chat_history
        })
    print(f'Response time: {time() - start_time} seconds')
    
    return result

def alert_human(str) -> str:
    return "[ALERT HUMAN]"

tools = [
    Tool(
        name=f"alert_human_function",
        func=alert_human,
        description=f"A function to use when you do not know how to respond to the InboundMessage because it cannot be answered from your instructions or documentation.",
    )
]

conversation_id = 4
InboundMessage = "I have a herniated disc. Can you still coach me?"
reply_dict[conversation_id] = dict()
question_id = 1

contactId = contacts['me_mcloone']
system_message_dict[conversation_id] = create_system_message(
    'CoachMcloone', prompts_filepath='../prompts',
    examples_filepath='../data/chat_examples', doc_filepath='../data/rag_docs'
)
conversation_dict[conversation_id] = create_chatbot(
    contactId, system_message_dict[conversation_id], tools=tools,
    # model='gpt-4-32k'
    )

reply_dict[conversation_id][question_id] = chat_with_chatbot(
    InboundMessage, conversation_dict[conversation_id]
)
reply_text = reply_dict[conversation_id][question_id]["output"]
print(f'Reply from `chat_with_chatbot`: {reply_text}')


NotImplementedError: 

# *End of Page*